In [ ]:
from ml_project.utils import files,paths,data
import geopandas as gpd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Correct Country = NaN

points_df = files.load_points_df()

samples_df = files.load_samples_df()

samples_df['country'].isna().sum()

data.EXCLUDED_COUNTRIES = []
countries_eu_110m = files.load_ne_countries_eu_110m()

check if middle is within coarse country border

samples_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(samples_df['lon'], samples_df['lat']))
for i, country in tqdm(countries_eu_110m.iterrows()):
    not_assigned = samples_df['country'].isna()
    in_country = not_assigned & samples_points.loc[not_assigned].within(country['geometry'])
    samples_df.loc[in_country,'country'] = country['NAME_EN']

samples_df['country'].isna().sum()

check if tile intersects with coarse country border

for i, country in tqdm(countries_eu_110m.iterrows()):
    not_assigned = samples_df['country'].isna()
    in_country = not_assigned & samples_df.loc[not_assigned].intersects(country['geometry'])
    samples_df.loc[in_country,'country'] = country['NAME_EN']

samples_df['country'].isna().sum()

samples_df.shape

files.save_samples_df(samples_df,override=True)

# points_df drop not in europe

points_df = files.load_points_df()

points_df.drop(index = points_df[(~points_df['in_eu'])].index,inplace=True)

files.save_points_df(points_df,override=True)

# drop duplicate samples

samples_df = files.load_samples_df()

# total
samples_df.shape

# duplicates
samples_df[samples_df.duplicated(subset=['tile_x','tile_y'],keep='first')].shape

# without duplicates
new_samples_df = samples_df.drop(samples_df[samples_df.duplicated(subset=['tile_x','tile_y'],keep='first')].index)
new_samples_df.shape

files.save_samples_df(new_samples_df,override=True)

# drop duplicated points with same id (how??)

points_df = files.load_points_df()

samples_df = files.load_samples_df()

points_df.shape

samples_df.shape

points_df.set_index('id').loc[samples_df['id']]

points_df.loc[points_df.duplicated(['id','tile_x','tile_y','tile_z'],keep='first')].shape

points_df.drop(points_df.loc[points_df.duplicated(['id','tile_x','tile_y','tile_z'],keep='first')].index, inplace=True)

files.save_points_df(points_df,override=True)

# new_tile for duplicate tiles to False

points_df = files.load_points_df()

points_df.shape

#duplicated
points_df[((points_df.duplicated(subset=['tile_x','tile_y','tile_z'],keep='first'))&(points_df['new_tile']))].shape

points_df.loc[(points_df.duplicated(subset=['tile_x','tile_y','tile_z'],keep='first'))&(points_df['new_tile']),'new_tile'] = False

files.save_points_df(points_df,override=True)

# Correct the Order of the columns

files.save_points_df(files.load_points_df(),override=True)

files.save_samples_df(files.load_samples_df(),override=True)

# Add filenames

samples_df = files.load_samples_df()

for i,sample in samples_df.iterrows():
    samples_df.loc[i,'satellite_file'] = paths.FILE_SATELLITE_IMAGE(sample['tile_x'],sample['tile_y'],sample['tile_z']).name
    samples_df.loc[i,'mask_file'] = paths.FILE_MASK_IMAGE(sample['tile_x'],sample['tile_y'],sample['tile_z']).name

files.save_samples_df(samples_df,override=True)

# Update filenames

samples_df = files.load_samples_df()

for i,sample in tqdm(samples_df.iterrows()):
    x = sample['tile_x']
    y = sample['tile_y']
    z = sample['tile_z']
    
    old_satellite = paths.SATELLITE_IMAGES / sample['satellite_file']
    old_mask = paths.MASK_IMAGES / sample['mask_file']
    
    new_satellite = paths.FILE_SATELLITE_IMAGE(x,y,z)
    new_mask = paths.FILE_MASK_IMAGE(x,y,z)
    
    old_satellite.rename(new_satellite)
    old_mask.rename(new_mask)
    
    samples_df.loc[i,'satellite_file'] = new_satellite.name
    samples_df.loc[i,'mask_file'] = new_mask.name

files.save_samples_df(samples_df,override=True)

files.load_samples_df()